In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !pip3 install idiomatch
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip install jieba
!pip install nltk
!pip install transformers
!pip install sentencepiece
!pip install -U synonyms
!python -c "import synonyms"

In [ ]:
import spacy
# from idiomatch import Idiomatcher
nlp_en = spacy.load("en_core_web_sm")
import synonyms
from jieba import posseg
# idiomatcher = Idiomatcher.from_pretrained(nlp_en)

In [ ]:
from nltk.corpus import wordnet 
import jieba
import nltk
nltk.download('wordnet')

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")
tokenizer_en = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX")

In [ ]:
class Generalization:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path

    def idiom(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            doc = nlp_en(sent)
            en_idiom = idiomatcher.identify(doc)
            if en_idiom != []:
                print(en)
                print(zh)
                print(en_idiom)


    def pronoun(self):
        r = 1
        for en,zh in zip(self.en_li[11000:], self.zh_li[11000:]):
            if '他们' in zh and 'hey' not in en and 'heir' not in en and 'them' not in en and 'who' not in en and 'that' not in en: #and '所有' not in zh and '全部' not in zh and '全' not in zh and '都' not in zh and '一' not in zh and 'at all' not in en: #and "it's" not in en and "it comes" not in en and "that" not in en:
                # if en_tok_li.index('it') != 0 and "".join(tok.pos_ for tok in nlp_en(en_tok_li[en_tok_li.index('it')-1])) in ['VERB','ADV']:
                    
                seg_zh = posseg.cut(zh)
                noun_list = [tok.text for tok in nlp_en(en) if tok.pos_ in ['NOUN'] and tok.text != tok.lemma_]
                if noun_list != []:
                    # print(en)
                    # print(zh)
                    # print(noun_list)
                    # print()
                    with open(save_path,'a+') as q:
                        q.write(en+'\n'+zh+'\n'+str(noun_list)+'\n'+'他们:'+'\n')    ####这里要改！！！！！！！！！！！
                        print(r)
                        r += 1

    def hypernym(self):
        r = 1
        for en,zh in zip(self.en_li, self.zh_li):
            en_tok_li = [tok.text for tok in nlp_en(en)]
            zh_tok_li = jieba.cut(zh)
            zh2en_dict = {}
            for tok_zh in zh_tok_li:
                en_token = tokenizer(tok_zh, return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
                zh_text_trans = [tok.lemma_ for tok in nlp_en(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])][0]
                # print(zh_text_trans)
                zh2en_dict[zh_text_trans] = tok_zh
            
            print(r)
            r+=1
            for en_tok in en_tok_li:
                
                if wordnet.synsets(en_tok) != []:
                    # print(en_tok)
                    hypernyms = wordnet.synsets(en_tok)[0].hypernyms()
                    hyper_li = [h.lemma_names() for h in hypernyms]
                    if hyper_li != []:
                        hyper_li_final = []
                        hyper_str = '_'.join(hyper_li[0])
                        for ph in hyper_li[0]:
                            if '_' in ph:
                                new_ph = ph.replace('_',' ')
                                hyper_li_final.append(new_ph)
                            else:
                                hyper_li_final.append(ph)
                                
                    
                        for en2zh in zh2en_dict.keys():
                            if en2zh in hyper_li_final:
                                # print(en)
                                # print(zh)
                                # print(en_tok, zh2en_dict[en2zh])
                                # print()
                                with open(save_path,'a+') as q:
                                    q.write(en+'\n'+zh+'\n'+en_tok+':'+zh2en_dict[en2zh]+'\n')



    def hyponym(self):
        r = 1
        for en,zh in zip(self.en_li[142000:], self.zh_li[142000:]):
            en_tok_li = [tok.text for tok in nlp_en(en)]
            zh_tok_li = jieba.cut(zh)
            zh2en_dict = {}
            for tok_zh in zh_tok_li:
                en_token = tokenizer(tok_zh, return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
                zh_text_trans = [tok.lemma_ for tok in nlp_en(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])][0]
                # print(zh_text_trans)
                zh2en_dict[zh_text_trans] = tok_zh
            
            print(r)
            r+=1
            for en_tok in en_tok_li:
                # print(en_tok)
                
                if wordnet.synsets(en_tok) != []:
                    # print(en_tok)
                    hyponyms = wordnet.synsets(en_tok)[0].hyponyms()
                    hypo_li = [h.lemma_names() for h in hyponyms]
                    if hypo_li != []:
                        hypo_li_final = []
                        for ph in hypo_li[0]:
                            if '_' in ph:
                                new_ph = ph.replace('_',' ')
                                hypo_li_final.append(new_ph)
                            else:
                                hypo_li_final.append(ph)
                        # print(hypo_li_final)
                                
                    
                        for en2zh in zh2en_dict.keys():
                            if en2zh in hypo_li_final:
                                # print(en)
                                # print(zh)
                                # print(en_tok, zh2en_dict[en2zh])
                                # print()
                                with open(save_path,'a+') as q:
                                    q.write(en+'\n'+zh+'\n'+en_tok+':'+zh2en_dict[en2zh]+'\n')







spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/News/Bi-News.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/hyponym_News_new.txt'             
        
ptcl = Generalization(spath, save_path)

ptcl.hyponym()

In [ ]:
class Cleaning:
    def __init__(self,path,save_path,n):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        self.align_li = []
        self.n = n
        for i, sent in enumerate(text):
            if i%self.n == 0:
                self.en_li.append(sent.strip())
            elif i%self.n == 1:
                self.zh_li.append(sent.strip())
            elif i%self.n == 2:
                self.align_li.append(sent.strip())
        self.save_path = save_path

    def clean_duplicate(self):

        for en,zh,align_pair in zip(self.en_li, self.zh_li, self.align_li):
            en_part = align_pair.split(':')[0]
            zh_part = align_pair.split(':')[-1]

            zh_tok_li = jieba.cut(zh)
            zh_tok_li_rest = [zh_tok for zh_tok in zh_tok_li if zh_tok != zh_part]

            en_token = tokenizer_en(en_part, return_tensors="pt")
            generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
            zh_text_trans = tokenizer_en.batch_decode(generated_tokens, skip_special_tokens=True)[0]
            # print(en_part)
            # print(zh_text_trans)

            if len(zh_part) != 0 and len(zh_text_trans) != 0:
                score = synonyms.compare(zh_part,zh_text_trans,seg=True)
                if score >=0.7:
                    real_align_li = []
                    for zh_tok_r in zh_tok_li_rest:

                        if len(zh_text_trans) !=0 and len(zh_tok_r) != 0:
                            r = synonyms.compare(zh_text_trans,zh_tok_r,seg=True)
                            if zh_text_trans not in zh and r <= 0.7 and len(zh_part)>1 and '妈' not in zh and 'specif' not in en:
                                real_align = en_part+':'+zh_part
                                real_align_li.append(real_align)
                    for real in list(set(real_align_li)):
                        print(en)
                        print(zh)
                        print(real)
                        with open(self.save_path,'a+') as y:
                          y.write(en+'\n'+zh+'\n'+real+'\n')

                        





                


spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/to_be_cleaned/Subtitles.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/to_be_cleaned/Subtitles_new.txt'             
        
ptcl = Cleaning(spath, save_path,3)

ptcl.clean_duplicate()
